# 1. Handling Outliers

In [1]:
import os
import pandas as pd

In [2]:
DATA_PATH = '../../data/'
train = pd.read_csv(os.path.join(DATA_PATH, 'train_1.csv'), index_col=0)
# test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

train_origin = train.copy()
# test_origin = test.copy()

train.head()

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,ID_-0.510_29.290_2019_00,-0.51,29.29,2019,0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,...,3664.436218,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140,3.750994
1,ID_-0.510_29.290_2019_01,-0.51,29.29,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,...,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779,4.025176
2,ID_-0.510_29.290_2019_02,-0.51,29.29,2019,2,0.000514,0.748199,0.000385,0.110018,72.795837,...,4216.986492,60068.894448,3516.282669,21.103410,0.251101,15.377883,30.401823,-142.519545,26.193296,4.231381
3,ID_-0.510_29.290_2019_03,-0.51,29.29,2019,3,0.000218,0.712247,0.000169,0.115591,38.458553,...,5228.507736,51064.547339,4180.973322,15.386899,0.262043,-11.293399,24.380357,-132.665828,28.829155,4.305286
4,ID_-0.510_29.290_2019_04,-0.51,29.29,2019,4,-0.000079,0.676296,-0.000048,0.121164,4.121269,...,3980.598120,63751.125781,3355.710107,8.114694,0.235847,38.532263,37.392979,-141.509805,22.204612,4.347317


# 1) 이상치 처리
- IQR 기반 이상치 확인
- flow
    1. 칼럼별 IQR 기반 이상치 탐지
    2. flooring / capping으로 이상치 처리

In [30]:
# train 칼럼들의 dtype 확인
train.dtypes.unique()

array([dtype('O'), dtype('float64'), dtype('int64')], dtype=object)

In [31]:
# numeric 칼럼 추출
numeric_columns = train.select_dtypes(exclude=['object']).columns
numeric_columns[:10]

Index(['latitude', 'longitude', 'year', 'week_no',
       'SulphurDioxide_SO2_column_number_density',
       'SulphurDioxide_SO2_column_number_density_amf',
       'SulphurDioxide_SO2_slant_column_number_density',
       'SulphurDioxide_cloud_fraction', 'SulphurDioxide_sensor_azimuth_angle',
       'SulphurDioxide_sensor_zenith_angle',
       'SulphurDioxide_solar_azimuth_angle',
       'SulphurDioxide_solar_zenith_angle',
       'SulphurDioxide_SO2_column_number_density_15km',
       'CarbonMonoxide_CO_column_number_density',
       'CarbonMonoxide_H2O_column_number_density',
       'CarbonMonoxide_cloud_height', 'CarbonMonoxide_sensor_altitude',
       'CarbonMonoxide_sensor_azimuth_angle',
       'CarbonMonoxide_sensor_zenith_angle',
       'CarbonMonoxide_solar_azimuth_angle',
       'CarbonMonoxide_solar_zenith_angle',
       'NitrogenDioxide_NO2_column_number_density',
       'NitrogenDioxide_tropospheric_NO2_column_number_density',
       'NitrogenDioxide_stratospheric_NO2_colum

In [55]:
# 이상치 수정 함수
def replace_outliers(x, upper_limit, lower_limit):
    if x > upper_limit:
        return upper_limit
    elif x < lower_limit:
        return lower_limit
    else:
        return x

In [61]:
outlier_columns = []
for col in numeric_columns:
    q1 = train[col].quantile(0.25)
    q3 = train[col].quantile(0.75)
    iqr = q3 - q1
    upper_limit = q3 + 1.5 * iqr
    lower_limit = q1 - 1.5 * iqr

    train[col] = train[col].apply(replace_outliers, args=[upper_limit, lower_limit])

    # 이상치가 제대로 제거되었는지 확인하기 위한 코드
    isin_outliers = len(train.loc[(train[col] > upper_limit) | (train[col] < lower_limit)])
    if isin_outliers:
        outlier_columns.append(col)

print(f'이상치가 존재하는 칼럼 개수: {len(outlier_columns)}')

이상치가 존재하는 칼럼 개수: 0


In [62]:
# 데이터 저장
data_name = os.path.join(DATA_PATH, 'train_2.csv')
train.to_csv(data_name)